In [1]:
import cv2
import glob
import numpy as np
from skimage.feature import graycomatrix, graycoprops
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
import matplotlib.pyplot as plt
import seaborn as sns  # Para la visualización de la matriz de confusión

# Definir las carpetas de frutas frescas y podridas
freshfruits = {
    "freshapples": "freshapple",
    "freshbanana": "freshbanana",
    "freshoranges": "freshorange"
}

rottenfruits = {
    "rottenapples": "rottenapple",
    "rottenbanana": "rottenbanana",
    "rottenoranges": "rottenorange"
}

# Inicializar listas para almacenar características y etiquetas
data_train = []
labels_train = []
data_test = []
labels_test = []

# Función para extraer Hu Moments
def extract_hu_moments(binary_img):
    moments = cv2.moments(binary_img)
    hu_moments = cv2.HuMoments(moments).flatten()
    # Log-transform para normalizar los valores
    hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-10)
    return hu_moments

# Función para extraer características GLCM
def extract_glcm_features(gray_img):
    # Definir distancias y ángulos para GLCM
    distances = [1]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    glcm = graycomatrix(gray_img, distances=distances, angles=angles, symmetric=True, normed=True)
    
    # Extraer propiedades de la GLCM
    contrast = graycoprops(glcm, 'contrast').flatten()
    dissimilarity = graycoprops(glcm, 'dissimilarity').flatten()
    homogeneity = graycoprops(glcm, 'homogeneity').flatten()
    energy = graycoprops(glcm, 'energy').flatten()
    correlation = graycoprops(glcm, 'correlation').flatten()
    
    # Concatenar todas las propiedades en un solo vector
    glcm_features = np.concatenate([contrast, dissimilarity, homogeneity, energy, correlation])
    return glcm_features

# Función para procesar imágenes y extraer características Hu Moments y GLCM
def process_images(folder_dict, base_path, data_list, labels_list):
    for folder, label in folder_dict.items():
        folder_path = f"{base_path}/{folder}/*.png"
        for image_path in glob.glob(folder_path):
            img = cv2.imread(image_path)
            if img is None or len(img.shape) != 3 or img.shape[2] != 3:
                print(f"Imagen no válida o con canales incorrectos: {image_path}")
                continue
            try:
                # Redimensionar la imagen
                resized_img = cv2.resize(img, (64, 32))
                
                # Conversión a escala de grises
                gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
                
                # Binarización de la imagen para Hu Moments
                _, binary_img = cv2.threshold(gray_img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
                
                # Extracción de Hu Moments
                hu_moments = extract_hu_moments(binary_img)
                
                # Extracción de características GLCM
                glcm_features = extract_glcm_features(gray_img)
                
                # Concatenar Hu Moments y GLCM
                features = np.concatenate([hu_moments, glcm_features])
            except Exception as e:
                print(f"Error procesando {image_path}: {e}")
                continue

            # Agregar las características y la etiqueta a las listas
            data_list.append(features)
            labels_list.append(label)

# Procesar imágenes de entrenamiento
train_base_path = ".gitignore/dataset/train"
process_images(freshfruits, train_base_path, data_train, labels_train)
process_images(rottenfruits, train_base_path, data_train, labels_train)

print("Total de imágenes de entrenamiento procesadas:", len(data_train))

# Procesar imágenes de prueba
test_base_path = ".gitignore/dataset/test"
process_images(freshfruits, test_base_path, data_test, labels_test)
process_images(rottenfruits, test_base_path, data_test, labels_test)

print("Total de imágenes de test procesadas:", len(data_test))

# Convertir las características y etiquetas a arrays de NumPy
X_train = np.array(data_train)
y_train_labels = np.array(labels_train)
X_test = np.array(data_test)
y_test_labels = np.array(labels_test)

# Codificar las etiquetas de forma consistente
encoder = LabelEncoder()
encoder.fit(np.concatenate((y_train_labels, y_test_labels)))

y_train = encoder.transform(y_train_labels)
y_test = encoder.transform(y_test_labels)

Total de imágenes de entrenamiento procesadas: 10901
Total de imágenes de test procesadas: 2698


In [3]:
# SVM amb Kernel Polinòmic - Model de Classificació
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Definir i entrenar el model SVM amb kernel polinòmic
poly_classifier = SVC(kernel='poly', degree=3, coef0=1, probability=True, random_state=42)
poly_classifier.fit(X_train, y_train)

# Predir sobre el conjunt de prova
y_pred_poly = poly_classifier.predict(X_test)

# Avaluar el model
accuracy_poly = accuracy_score(y_test, y_pred_poly)
print("----- SVM amb Kernel Polinòmic -----")
print(f"Accuracy: {accuracy_poly:.4f}\n")
print("Informe de classificació:")
print(classification_report(y_test, y_pred_poly, target_names=encoder.classes_))
print("Matriu de Confusió:")
print(confusion_matrix(y_test, y_pred_poly))

----- SVM amb Kernel Polinòmic -----
Accuracy: 0.3462

Informe de classificació:
              precision    recall  f1-score   support

  freshapple       0.36      0.12      0.18       395
 freshbanana       0.41      0.38      0.40       381
 freshorange       0.35      0.02      0.04       388
 rottenapple       0.28      0.86      0.42       601
rottenbanana       0.69      0.39      0.50       530
rottenorange       0.44      0.02      0.03       403

    accuracy                           0.35      2698
   macro avg       0.42      0.30      0.26      2698
weighted avg       0.42      0.35      0.29      2698

Matriu de Confusió:
[[ 48  36   6 290  15   0]
 [ 15 145   2 188  26   5]
 [  7  71   9 294   6   1]
 [ 22  29   0 517  33   0]
 [ 33  25   0 261 208   3]
 [  8  47   9 318  14   7]]
